In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
num_epochs = 5
batch_size = 100
learning_rate = 0.001

transform = transforms.Compose([
    transforms.ToTensor(), # Converts image to PyTorch Tensor
    transforms.Normalize((0.1307,), (0.3081,)) # Normalize with MNIST's mean and std
])

In [4]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 9.91M/9.91M [01:38<00:00, 101kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 59.2kB/s]
100%|██████████| 1.65M/1.65M [00:23<00:00, 70.4kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 277kB/s]


In [5]:
class BasicBlock(nn.Module):
    expansion = 1 # No expansion of channels in the basic block

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Shortcut connection to match dimensions if they change (e.g., due to stride)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Add the shortcut connection's output
        out += self.shortcut(identity)
        out = self.relu(out)
        return out


In [6]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        # Initial convolution (adapted for MNIST's 1 channel)
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        
        # Stacking residual blocks
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        
        # Final layers
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for s in strides:
            layers.append(block(self.in_channels, out_channels, s))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

In [7]:
model = ResNet(BasicBlock, [2, 2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
print("\nStarting Training... 🚀")
total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 200 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}')



Starting Training... 🚀
Epoch [1/5], Step [200/600], Loss: 0.0817
Epoch [1/5], Step [400/600], Loss: 0.0337
Epoch [1/5], Step [600/600], Loss: 0.1192
Epoch [2/5], Step [200/600], Loss: 0.0210
Epoch [2/5], Step [400/600], Loss: 0.0248
Epoch [2/5], Step [600/600], Loss: 0.0810
Epoch [3/5], Step [200/600], Loss: 0.0885
Epoch [3/5], Step [400/600], Loss: 0.0951
Epoch [3/5], Step [600/600], Loss: 0.0082
Epoch [4/5], Step [200/600], Loss: 0.0208
Epoch [4/5], Step [400/600], Loss: 0.0059
Epoch [4/5], Step [600/600], Loss: 0.0122
Epoch [5/5], Step [200/600], Loss: 0.0640
Epoch [5/5], Step [400/600], Loss: 0.0027
Epoch [5/5], Step [600/600], Loss: 0.0395


In [9]:
print("\nEvaluating Model...")
model.eval()  # Set model to evaluation mode
with torch.no_grad(): # Disable gradient calculation
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the 10000 test images: {accuracy:.2f} %')


Evaluating Model...
Accuracy of the model on the 10000 test images: 99.01 %
